# Strategy Exploration Notebook

This notebook demonstrates how to:
1. Load stock data
2. Create and test trading strategies
3. Run backtests
4. Visualize results

In [ ]:
# Import required modules
import pandas as pd
import numpy as np
from raj.data.provider import DataProvider
from raj.strategies.examples.buy_hold import BuyAndHoldStrategy
from raj.backtest.engine import BacktestEngine
from raj.backtest.metrics import PerformanceMetrics
from raj.visualization.charts import ChartGenerator

## 1. Load Data

In [ ]:
# Create data provider
provider = DataProvider()

# Load top 20 stocks
data = provider.get_universe_data(
    universe_name='top20',
    start_date='2020-01-01'
)

print(f"Loaded {len(data)} symbols")
print(f"Symbols: {', '.join(sorted(data.keys()))}")

## 2. Explore a Single Stock

In [ ]:
# Pick a stock to explore
symbol = list(data.keys())[0]
stock_data = data[symbol]

print(f"\n{symbol} Data:")
print(stock_data.head())
print(f"\nDate range: {stock_data.index.min().date()} to {stock_data.index.max().date()}")
print(f"Number of rows: {len(stock_data)}")

In [ ]:
# Plot price chart
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=stock_data.index,
    open=stock_data['open'],
    high=stock_data['high'],
    low=stock_data['low'],
    close=stock_data['close'],
    name=symbol
))
fig.update_layout(
    title=f"{symbol} - Price History",
    yaxis_title="Price ($)",
    template='plotly_white'
)
fig.show()

## 3. Create and Test a Strategy

In [ ]:
# Create a simple buy-and-hold strategy
strategy = BuyAndHoldStrategy()
print(f"Strategy: {strategy}")

In [ ]:
# Generate signals for a single stock
signals = strategy.generate_signals(stock_data)
print(f"\nSignals generated: {len(signals)}")
print(f"\nSignal distribution:")
print(signals.value_counts())

In [ ]:
# Visualize signals on price chart
fig = ChartGenerator.plot_signals(stock_data, signals, symbol)
fig.show()

## 4. Run Backtest

In [ ]:
# Create backtest engine
engine = BacktestEngine(
    initial_capital=100_000,
    commission=0.001
)

# Run backtest on all symbols
result = engine.run(strategy, data)

# Print metrics
PerformanceMetrics.print_metrics(result.metrics)

## 5. Visualize Results

In [ ]:
# Create performance dashboard
dashboard = ChartGenerator.create_dashboard(result)
dashboard.show()

In [ ]:
# Plot equity curve
equity_fig = ChartGenerator.plot_equity_curve(result.equity_curve)
equity_fig.show()

In [ ]:
# Plot drawdown
drawdown_fig = ChartGenerator.plot_drawdown(result.equity_curve)
drawdown_fig.show()

## 6. Analyze Trades

In [ ]:
# Convert trades to DataFrame for analysis
if len(result.trades) > 0:
    trades_df = pd.DataFrame([{
        'symbol': t.symbol,
        'entry_date': t.entry_date,
        'exit_date': t.exit_date,
        'entry_price': t.entry_price,
        'exit_price': t.exit_price,
        'quantity': t.quantity,
        'pnl': t.pnl,
        'pnl_pct': t.pnl_pct,
        'hold_days': (t.exit_date - t.entry_date).days
    } for t in result.trades])
    
    print(f"Total trades: {len(trades_df)}")
    print(f"\nTop 10 trades by P&L:")
    print(trades_df.nlargest(10, 'pnl')[['symbol', 'pnl', 'pnl_pct', 'hold_days']])
    
    print(f"\nBottom 10 trades by P&L:")
    print(trades_df.nsmallest(10, 'pnl')[['symbol', 'pnl', 'pnl_pct', 'hold_days']])
else:
    print("No trades executed")

## 7. Create Your Own Strategy

You can create custom strategies by inheriting from `BaseStrategy`:

In [ ]:
from raj.strategies.base import BaseStrategy
from raj.backtest.signals import SignalType

class SimpleMovingAverageCrossover(BaseStrategy):
    """Simple moving average crossover strategy."""
    
    def __init__(self, fast_period=50, slow_period=200):
        super().__init__(fast_period=fast_period, slow_period=slow_period)
    
    def generate_signals(self, data: pd.DataFrame) -> pd.Series:
        """Generate signals based on MA crossover."""
        # Calculate moving averages
        fast_ma = data['close'].rolling(self.parameters['fast_period']).mean()
        slow_ma = data['close'].rolling(self.parameters['slow_period']).mean()
        
        # Initialize signals
        signals = pd.Series(SignalType.HOLD, index=data.index)
        
        # Generate buy signals when fast MA crosses above slow MA
        signals[(fast_ma > slow_ma) & (fast_ma.shift(1) <= slow_ma.shift(1))] = SignalType.BUY
        
        # Generate sell signals when fast MA crosses below slow MA
        signals[(fast_ma < slow_ma) & (fast_ma.shift(1) >= slow_ma.shift(1))] = SignalType.SELL
        
        return signals

# Test the custom strategy
custom_strategy = SimpleMovingAverageCrossover(fast_period=50, slow_period=200)
print(f"Created custom strategy: {custom_strategy}")

In [ ]:
# Run backtest with custom strategy
custom_result = engine.run(custom_strategy, data)
PerformanceMetrics.print_metrics(custom_result.metrics)

In [ ]:
# Compare strategies
comparison = pd.DataFrame({
    'Buy & Hold': [
        result.metrics['total_return'],
        result.metrics['sharpe_ratio'],
        result.metrics['max_drawdown'],
        result.metrics['num_trades']
    ],
    'MA Crossover': [
        custom_result.metrics['total_return'],
        custom_result.metrics['sharpe_ratio'],
        custom_result.metrics['max_drawdown'],
        custom_result.metrics['num_trades']
    ]
}, index=['Total Return (%)', 'Sharpe Ratio', 'Max Drawdown (%)', 'Number of Trades'])

print("\nStrategy Comparison:")
print(comparison)